In [3]:


import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers

import pickle
import numpy as np

# f2 = open('data/split_image_0.60_0.30_0.10_aug_to_1875_down_sample_True_instance0-9/val_data', 'rb')
f2 = open('data/split_0.60_0.30_0.10_aug_to_1875_down_sample_False_instance0-9/val_data', 'rb')
# X_sequence_val, X_feature_val, X_image_val, Y_val = pickle.load(f2)
X_sequence_val, X_feature_val, Y_val = pickle.load(f2)
f2.close()



from keras_preprocessing.sequence import pad_sequences
from util import preprocess

total_X_sequence_train = []
total_X_feature_train = []
# total_X_image_train = []
total_Y_train = []
def build_channel(data):
    X = []
    for image in data:
        X.append(image.reshape(128, 128, 1))
    return X
for i in range(0,10):
    # f = open('data/split_image_0.60_0.30_0.10_aug_to_1875_down_sample_True_instance0-9/train_data%d'%i, 'rb')
    f = open('data/split_0.60_0.30_0.10_aug_to_1875_down_sample_False_instance0-9/train_data%d'%i, 'rb')
    # X_sequence_train, X_feature_train, X_image_train, Y_train = pickle.load(f)
    X_sequence_train, X_feature_train, Y_train = pickle.load(f)
    X_sequence_train_paded = pad_sequences(X_sequence_train, dtype='float', padding='post', value = np.nan)
    X_sequence_train_trimed = preprocess(X_sequence_train_paded)
    X_feature_train = np.array(X_feature_train)
    # X_image_train = np.array(build_channel(X_image_train))
    Y_train = np.array(Y_train)-1
    Y_train = keras.utils.to_categorical(Y_train, num_classes=11)

    total_X_sequence_train.append(X_sequence_train_trimed)
    total_X_feature_train.append(X_feature_train)
    # total_X_image_train.append(X_image_train)
    total_Y_train.append(Y_train)
    f.close()

X_sequence_val_paded = pad_sequences(X_sequence_val, dtype='float', padding='post', value = np.nan)
X_sequence_val_trimed = preprocess(X_sequence_val_paded)
X_feature_val = np.array(X_feature_val)
# X_image_val = np.array(build_channel(X_image_val))
Y_val = np.array(Y_val)-1


Y_val = keras.utils.to_categorical(Y_val, num_classes=11)

In [4]:
f = open('data/split_0.60_0.30_0.10_aug_to_1875_down_sample_False_instance0-9/class_weights0', 'rb')
train_weight, test_weight = pickle.load(f)
f.close()
print(train_weight)

[0.72226502 0.83259325 1.         1.         0.16617921 0.69264869
 0.85890976 1.         1.         1.         1.        ]


In [5]:
from mix_models import rnn_with_feature

X_val = {'sequence':X_sequence_val_trimed, 'feature':X_feature_val}
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-3,
    decay_steps=860,
    decay_rate=0.85)

for i in range(0,10):
    model = rnn_with_feature(sequence_shape=(None, 3),feature_shape=(2,), num_classes=11)
    model.compile(
    optimizer=keras.optimizers.Adam(1e-3),
    # loss="sparse_categorical_crossentropy",
    loss=keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
    # metrics=[keras.metrics.SparseCategoricalAccuracy(name="acc")],
    metrics=[keras.metrics.CategoricalAccuracy(name="acc")],
    run_eagerly=False,
    )
    callbacks = [
    keras.callbacks.ModelCheckpoint(
        # filepath='models/bagging_before_image_0.60_0.30_0.10/rnn_with_feature%d_extra/model_{epoch}'%i,
        filepath='models/10x_0.60_0.30_0.10/rnn_with_feature%d_extra/model_{epoch}'%i,
        save_freq='epoch'),
    keras.callbacks.TensorBoard(
        # log_dir='models/bagging_before_image_0.60_0.30_0.10/rnn_with_feature%d_extra/logs'%i
        log_dir='models/10x__0.60_0.30_0.10/rnn_with_feature%d_extra/logs'%i
        )
    ]
    X_train = {'sequence':total_X_sequence_train[i], 'feature':total_X_feature_train[i]}
    model.fit(X_train, total_Y_train[i], batch_size=32, epochs=10, 
        class_weight=dict(enumerate(train_weight)),
        validation_data=(X_val, Y_val), callbacks=callbacks)


2023-03-29 13:25:01.384685: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-29 13:25:02.915528: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22216 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:73:00.0, compute capability: 8.6
2023-03-29 13:25:02.919174: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 22277 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:d5:00.0, compute capability: 8.6


Epoch 1/10


2023-03-29 13:25:13.037791: W tensorflow/core/common_runtime/forward_type_inference.cc:332] Type inference failed. This indicates an invalid graph that escaped type checking. Error message: INVALID_ARGUMENT: expected compatible input types, but input 1:
type_id: TFT_OPTIONAL
args {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_TENSOR
    args {
      type_id: TFT_FLOAT
    }
  }
}
 is neither a subtype nor a supertype of the combined inputs preceding it:
type_id: TFT_OPTIONAL
args {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_TENSOR
    args {
      type_id: TFT_INT32
    }
  }
}

	while inferring type of node 'cond_43/output/_16'
2023-03-29 13:25:14.179884: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8401
2023-03-29 13:25:14.180035: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
Could not load symbol cublasGetSmCountTarget from libcublas.so.11. Error: /ho

1009/1009 [==============================] - ETA: 0s - loss: 1.2288 - acc: 0.3441

INFO:tensorflow:Assets written to: models/10x_0.60_0.30_0.10/rnn_with_feature0_extra/model_1/assets


...

INFO:tensorflow:Assets written to: models/10x_0.60_0.30_0.10/rnn_with_feature9_extra/model_9/assets


1009/1009 [==============================] - 113s 112ms/step - loss: 0.4609 - acc: 0.8799 - val_loss: 1.0476 - val_acc: 0.8115
Epoch 10/10
1009/1009 [==============================] - ETA: 0s - loss: 0.4489 - acc: 0.8814

INFO:tensorflow:Assets written to: models/10x_0.60_0.30_0.10/rnn_with_feature9_extra/model_10/assets


INFO:tensorflow:Assets written to: models/10x_0.60_0.30_0.10/rnn_with_feature9_extra/model_10/assets


1009/1009 [==============================] - 115s 114ms/step - loss: 0.4489 - acc: 0.8814 - val_loss: 1.0878 - val_acc: 0.7875


In [ ]:
from mix_models import cnn_with_feature_3

X_val = {'sequence':X_sequence_val_trimed, 'feature':X_feature_val, 'image':X_image_val}
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-3,
    decay_steps=860,
    decay_rate=0.85)

for i in range(0,10):
    model = cnn_with_feature_3(feature_shape=(2,),image_shape=(128, 128, 1), num_classes=11)
    model.compile(
    optimizer=keras.optimizers.Adam(1e-3),
    # loss="sparse_categorical_crossentropy",
    loss=keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
    # metrics=[keras.metrics.SparseCategoricalAccuracy(name="acc")],
    metrics=[keras.metrics.CategoricalAccuracy(name="acc")],
    run_eagerly=False,
    )
    callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath='models/bagging_only_image_0.60_0.30_0.10/cnn_with_feature%d/model_{epoch}'%i,
        save_freq='epoch'),
    keras.callbacks.TensorBoard(log_dir='models/bagging_only_image_0.60_0.30_0.10/cnn_with_feature%d/logs'%i)
    ]
    X_train = {'sequence':total_X_sequence_train[i], 'feature':total_X_feature_train[i], 'image':total_X_image_train[i]}
    model.fit(X_train, total_Y_train[i], batch_size=32, epochs=100, 
        validation_data=(X_val, Y_val), callbacks=callbacks)

In [ ]:
best_acc_list = [6,5,4,3,5,8,5,4,7,8]
min_loss_list = [6,6,4,8,7,8,5,6,5,8]
for i in range(0,10):
    loaded_model = keras.models.load_model('models/bagging_before_image_0.60_0.30_0.10/\
rnn_with_feature%d/model_%d'%(i,min_loss_list[i]), compile=False)
    loaded_model.save_weights('models/bagging_before_image_weights/rnn_weights_min_loss%d.h5'%i)

In [2]:
from mix_models import multi_input_model
X_val = {'sequence':X_sequence_val_trimed, 'feature':X_feature_val, 'image':X_image_val}
for i in range(0,10):
    model = multi_input_model(sequence_shape=(None, 3), image_shape=(128, 128, 1), 
                        feature_shape=(2,), num_classes=11, cnn_lr_lamd=10)
    model.compile(
    optimizer=keras.optimizers.Adam(1e-3),

    loss=keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
    metrics=[keras.metrics.CategoricalAccuracy(name="acc")],
    run_eagerly=False,
    )
    callbacks = [
    keras.callbacks.ModelCheckpoint(
      filepath='models/bagging_after_image_0.60_0.30_0.10/mix_model%d/model_{epoch}'%i,
      save_freq='epoch'),
    keras.callbacks.TensorBoard(log_dir='models/bagging_after_image_0.60_0.30_0.10/mix_model%d/logs'%i)
    ]
    X_train = {'sequence':total_X_sequence_train[i], 'feature':total_X_feature_train[i],
              'image':total_X_image_train[i]}
    model.load_weights('models/bagging_before_image_weights/rnn_weights_min_loss%d.h5'%i, by_name=True)
    model.fit(X_train, total_Y_train[i], batch_size=32, epochs=100, 
        validation_data=(X_val, Y_val), callbacks=callbacks)


2023-02-05 17:45:45.279292: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-05 17:45:46.115948: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22231 MB memory:  -> device: 0, name: GeForce RTX 3090, pci bus id: 0000:73:00.0, compute capability: 8.6
2023-02-05 17:45:46.117034: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 22284 MB memory:  -> device: 1, name: GeForce RTX 3090, pci bus id: 0000:d5:00.0, compute capability: 8.6


Epoch 1/100


2023-02-05 17:45:57.450006: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-02-05 17:45:57.602104: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8401
2023-02-05 17:45:58.898656: W tensorflow/stream_executor/gpu/asm_compiler.cc:111] *** WARNING *** You are using ptxas 9.1.108, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2023-02-05 17:45:58.901264: W tensorflow/stream_executor/gpu/asm_compiler.cc:230] Falling back to the CUDA driver for PTX compilation; ptxas does not support CC 8.6
2023-02-05 17:45:58.901285: W tensorflow/stream_executor/gpu/asm_compiler.cc:233] Used ptxas at ptxas
2023-02-05 17:45:58.901334: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] UNIMPLEMENTED: 

645/645 [==============================] - ETA: 0s - loss: 2.4389 - acc: 0.7410

KeyboardInterrupt: 